In [1]:
# Add local scripts to module search path
import sys
import os
sys.path.append(os.path.realpath('../toliman-proper'))

# FFT Comparison for Toliman modelling

PROPER supports Fast Fourier Transforms (FFTs) using Numpy's internal algorithms, or using external FFTW or Intel MKL libraries. This notebook compares the speed of executing a propagation of the Toliman system for each case.

This test uses the `form_detector_image` function used to wrap up the operations in PROPER to form a single detector image. Because of a race condition bug in the current implementation of PROPER for FFTW the test only considers the case of a single wavelength, whereas in practice one would expect to model for a spread of wavelengths. 

In [2]:
import timeit

# These parameters affect the sampling, and therefore the amount of FFT computation 
gridsize = 2048 # sampling of wavefront
beam_ratio = 0.4

# How many iterations to perform
nits = 10

results = {}

In [3]:
# Setup for image
from proper_tools import form_detector_image
from spirals import binarized_ringed

prescription = 'prescription_rc_quad'
# The actual values probably don't matter all that much for the purposes of this comparison.
toliman_settings = {
                    'diam': 0.001 * 2. * 150, 
                    'm1_fl': 571.7300 / 1000.,
                    'm1_m2_sep': 549.240/1000.,
                    'm2_fl': -23.3800/1000.,
                    'bfl': 590.000 / 1000., 
                    'm2_rad': 5.9 / 1000., 
                    'm2_strut_width': 0.01,
                    'm2_supports': 5,
                    'beam_ratio': beam_ratio,
                    'tilt_x': 0.00,
                    'tilt_y': 0.00,
                    'phase_func': binarized_ringed
                    }


detector_pitch = 11.0e-6 # m/pixel on detector
npixels = 512 # Size of detector, in pixels

# First source, on axis
source_a = {
            'wavelengths': [0.6],
            'weights': [1.],
            'settings': toliman_settings
            }

# Second source, off-axis
tilted = toliman_settings.copy()
tilted['tilt_x'] = 3.00
tilted['tilt_y'] = 1.00

source_b = {
            'settings': tilted,
            'wavelengths': [0.6],
            'weights': [1.]
            }

def timed_op():
    return form_detector_image(prescription, [source_a, source_b], gridsize, detector_pitch, npixels)

op_txt = 'timed_op()'

/suphys/bjef8061/.local/lib/python3.5/site-packages/pyfits-3.5-py3.5-linux-x86_64.egg/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


## Default Numpy FFT

In [4]:
import proper
proper.prop_use_fftw(DISABLE=True)

In [7]:
result = timeit.timeit(op_txt, number=nits, globals=globals())
print(result)
results['numpy'] = result

414.4576419549994


## FFTW library



In [10]:
import proper
proper.prop_use_fftw()
proper.prop_fftw_wisdom(gridsize)

'/suphys/bjef8061/.local/lib/python3.5/site-packages/PyPROPER3-1.0.0-py3.5-linux-x86_64.egg/proper/.2048pix_wisdomfile'

In [11]:
result = timeit.timeit(op_txt, number=nits, globals=globals())
print(result)
results['fftw'] = result

402.11712781200185


## Intel MKL

Intel Math Kernel Library

In [4]:
import proper
proper.prop_use_ffti(MKL_DIR='/usr/physics/ic16/mkl/lib/intel64/')

In [5]:
result = timeit.timeit(op_txt, number=nits, globals=globals())
print(result)
results['intel'] = result

352.7674847957678


## Results

In [7]:
print(results)

{'numpy': 414.4576419549994, 'fftw': 402.11712781200185, 'intel': 352.7674847957678}
